In [27]:
import cv2
import numpy as np
import glob
import pickle
import os
import tqdm
import matplotlib.pyplot as plt

In [28]:
def get_layout(layout: str) -> None:
    """Initialize the layout of candidates."""

    if layout == 'openpose':
        num_node = 18
        inward = [(4, 3), (3, 2), (7, 6), (6, 5), (13, 12), (12, 11),
                        (10, 9), (9, 8), (11, 5), (8, 2), (5, 1), (2, 1),
                        (0, 1), (15, 0), (14, 0), (17, 15), (16, 14)]
        center = 1
    elif layout == 'human3.6m':
        num_node = 17
        inward = [(10, 9), (9, 8), (8,7), (7,0), (13, 12), (12, 11), (11, 8),
                  (16, 15), (15, 14), (14, 8),
                  (13, 12), (12, 11), (11, 8),
                  (3, 2), (2,1), (1,0),
                  (6,5), (5,4), (4,0)]
        center = 0
    elif layout == 'nturgb+d':
        num_node = 25
        neighbor_base = [(1, 2), (2, 21), (3, 21), (4, 3), (5, 21), (6, 5),
                            (7, 6), (8, 7), (9, 21), (10, 9), (11, 10),
                            (12, 11), (13, 1), (14, 13), (15, 14), (16, 15),
                            (17, 1), (18, 17), (19, 18), (20, 19), (22, 8),
                            (23, 8), (24, 12), (25, 12)]
        inward = [(i - 1, j - 1) for (i, j) in neighbor_base]
        center = 21 - 1
    elif layout == 'coco':
        num_node = 17
        inward = [(15, 13), (13, 11), (16, 14), (14, 12), (11, 5),
                        (12, 6), (9, 7), (7, 5), (10, 8), (8, 6), (5, 0),
                        (6, 0), (1, 0), (3, 1), (2, 0), (4, 2)]
        center = 0
    elif isinstance(layout, dict):
        num_node = layout['num_node']
        inward = layout['inward']
        center = layout['center']
    else:
        raise ValueError(f'Do Not Exist This Layout: {layout}')
    return num_node, inward, center

In [29]:
get_name_from_dir = lambda x: os.path.splitext(x.split(os.sep)[-1])[0]

In [30]:
SUBJECT = "D03"
vids = glob.glob(f"./Data/{SUBJECT}_raw/*.mp4")
#vid_names = 
with open("./Data/InfAct_plus/2d/primitive/InfAct_plus.pkl", "rb") as f:
    DS = pickle.load(f)

annotations = [item for item in DS['annotations'] if SUBJECT in item['source']]

In [35]:
a = [item for item in DS['annotations'] if "130851" in item['frame_dir']]
a[0]

{'pos_number': 2,
 'start_frame': 132,
 'end_frame': 400,
 'frame_dir': 'D03_nooie_20230117-130851@300000_Posture 2',
 'total_frames': 268,
 'img_shape': (1080, 1920),
 'original_shape': (1080, 1920),
 'pos_label': 'Standing',
 'source': 'CAREER_D03',
 'keypoint': array([[[[ 516.6605835 ,  961.41448975],
          [ 437.38653564,  952.76416016],
          [1481.61340332,  952.76416016],
          ...,
          [ 976.56195068,  407.86373901],
          [-106.09091949, 1762.86877441],
          [2025.09094238, 1762.86877441]],
 
         [[ 516.6605835 ,  961.41448975],
          [ 437.38653564,  952.76416016],
          [1481.61340332,  952.76416016],
          ...,
          [ 976.56195068,  407.86373901],
          [-106.09091949, 1762.86877441],
          [2025.09094238, 1762.86877441]],
 
         [[ 516.6605835 ,  961.41448975],
          [ 437.38653564,  952.76416016],
          [1481.61340332,  952.76416016],
          ...,
          [ 976.56195068,  407.86373901],
          [-1

In [6]:
def add_skel_to_img(img, kps):
    Xs = kps[:,0].astype(int)
    Ys = kps[:,1].astype(int)
    _, inward, _ = get_layout('coco')
    img_cp = img.copy()
    for (x,y) in zip(Xs, Ys):
        img_cp = cv2.circle(img_cp, (x,y), radius=5, color=(0, 0, 0), thickness=-1)
    for bone in inward:
        img_cp = cv2.line(img_cp,(Xs[bone[0]], Ys[bone[0]]), (Xs[bone[1]], Ys[bone[1]]),(0, 140, 255),2)
    return img_cp

In [26]:
for vid_pth in tqdm.tqdm(vids[-5:-4]):
    vid_name = get_name_from_dir(vid_pth)
    print(vid_name)
    vid_annos = [item for item in annotations if vid_name in item['frame_dir']]
    starts = [item['start_frame'] for item in vid_annos]
    ends = [item['end_frame'] for item in vid_annos]
    poses = [item['pos_label'] for item in vid_annos]
    skels = [item['keypoint'] for item in vid_annos]

    clips = [[] for _ in range(len(vid_annos))]

    cap = cv2.VideoCapture(vid_pth)
    
    frame_count = 0
    clip_count = 0
    while cap.isOpened():
        
        if clip_count>=len(vid_annos):
             break
        
        ret,frame = cap.read()
        if frame_count==0:
            height, width, _= frame.shape

        if frame_count>=starts[clip_count] and frame_count<ends[clip_count]-1:
            try:
                skel = skels[clip_count][0,frame_count-starts[clip_count]]
                aug_frame = add_skel_to_img(frame, skel)
                clips[clip_count].append(aug_frame)
            except Exception as E:
                print(E)
                pass

        if frame_count==ends[clip_count]:
            clip_count+=1
            if clip_count==11:
                clip_count+=1

        frame_count = frame_count + 1

    print('Here')
    cap.release()
    cv2.destroyAllWindows()

    out_dir = os.path.join(f"Data/{SUBJECT}_clips/{vid_name}/")
    os.makedirs(out_dir, exist_ok=True)
    for i, clip in enumerate(clips):
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
        out_pth = os.path.join(out_dir,f"video_{vid_name}_{i}_{poses[i]}.avi")
        video = cv2.VideoWriter(out_pth, fourcc, 5, (width, height))

        for f in clip:
            video.write(f)

        video.release()
        cv2.destroyAllWindows()

  0%|          | 0/1 [00:00<?, ?it/s]

nooie_20230118-151749@300000
Here


100%|██████████| 1/1 [00:56<00:00, 56.89s/it]


In [24]:
for i, (s, e) in enumerate(zip(starts, ends)):
    print(i, s,e)

0 1 136
1 160 209
2 238 493
3 512 576
4 590 1588
5 1630 1696
6 1711 1735
7 1749 2144
8 2169 2227
9 2255 3124
10 3157 3176
11 3148 3158
12 3200 3320
13 3332 3708
14 4059 4070
15 4079 4450


In [15]:
len(ends)

16